In [1]:
import numpy as np
import copy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

from sumo_interface import start_sim

def random_dur(default_dur, sigma=1):
	default_dur = sum(default_dur,[])
	result =  np.clip(np.random.normal(default_dur,sigma*np.ones(len(default_dur)),len(default_dur)),0.5,60)
	print("generated: ",result)
	return result

def evaluation(file_path,id_TLs,dur_TLs_):
	dur_TLs = copy.deepcopy(dur_TLs_)
	print("eval ind: ",dur_TLs)
	edit_net(file_path,id_TLs,dur_TLs)
	#--> to sumo api


	start_sim("/home/lin/workspace/GA/GA_Traffic_Optimization/demo_sumo/osm.sumocfg")
	#start_sim("/home/zhenyuli/workspace/GA_Traffic_Optimization/demo_sumo/osm.sumocfg")
	import xml.etree.ElementTree as ET
	from numpy import mean
	tree = ET.ElementTree(file = "/home/lin/workspace/GA/GA_Traffic_Optimization/result.xml")
	#tree = ET.ElementTree(file = "/home/zhenyuli/workspace/GA_Traffic_Optimization/result.xml")
	trip_infos = tree.getroot()
	timeLoss = mean([float(trip.attrib['timeLoss']) for trip in trip_infos])

	print("##############################################################")
	print("##############################################################")
	print("avg time loss: ", timeLoss)
	print("##############################################################")
	print("##############################################################")
	return timeLoss,

def checkBounds(min, max):
    def decorator(func):
        def wrapper(*args, **kargs):
            offspring = func(*args, **kargs)
            for child in offspring:
                for i in range(len(child)):
                    if child[i] > max:
                        child[i] = max
                    elif child[i] < min:
                        child[i] = min
            return offspring
        return wrapper
    return decorator
'''
def printsel():
    def decorator(func):
        def wrapper(*args, **kargs):
            print(*args)
            result = func(*args, **kargs)
            print("selected",result)
            return result
        return wrapper
    return decorator
'''
def ga(file_path,default_dur,id_TLs):
	#parameters
	INIT_SIZE = 10
	MAX_ITER = 5
	SIGMA = 1
	P_CROSSOVER = 0.5
	P_MUTATION = 0.1
	MIN = 1
	MAX = 60

	#DEAP
	creator.create("FitnessMin",base.Fitness,weights=(-1.0,)) 
	creator.create("Individual",list,fitness=creator.FitnessMin) #individuals are returned in list

	toolbox = base.Toolbox()
	toolbox.register("attr_item", random_dur, default_dur, SIGMA)
	toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.attr_item)
	toolbox.register("population", tools.initRepeat, list, toolbox.individual)

	toolbox.register("evaluate", evaluation, file_path,id_TLs)
	toolbox.register("edit_net", edit_net,file_path,id_TLs,dur_TLs)
	toolbox.register("start_sim",start_sim,file_path)
	toolbox.register("mate", tools.cxOnePoint)
	#toolbox.register("mate", crossover)
	toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=SIGMA, indpb=0.1)
	toolbox.decorate("mutate", checkBounds(MIN, MAX))# Bounds are still needed to be set.

	toolbox.register("select", tools.selTournament, tournsize=5)
	#toolbox.decorate("select", printsel())

	pop = toolbox.population(n=INIT_SIZE)

	hof = tools.HallOfFame(1) #pick the best one
	stats = tools.Statistics(lambda ind: ind.fitness.values)

	pop,log = algorithms.eaSimple(pop,toolbox,cxpb=P_CROSSOVER,mutpb=P_MUTATION,ngen=MAX_ITER,halloffame=hof,verbose=True)

	return hof, stats, pop, log


def get_default_duration(file_path):
	import xml.etree.ElementTree as ET

	tree = ET.ElementTree(file = file_path)
	net = tree.getroot()
	trafficLights = net.findall("tlLogic")

	id_TLs = []
	dur_TLs = []
	for tl in trafficLights:
		id_TLs.append(tl.attrib['id'])
		durations = [phase.attrib['duration'] for phase in tl]
		dur_TLs.append(durations)

	return id_TLs,dur_TLs


def edit_net(file_path,id_TLs,dur_TLs):
	import xml.etree.ElementTree as ET

	tree = ET.ElementTree(file = file_path)
	net = tree.getroot()
	trafficLights = net.findall("tlLogic")

	tl_idx = [[tl.attrib['id'] for tl in trafficLights].index(id) for id in id_TLs]

	for i in tl_idx:
		nb_phase = len(trafficLights[i])
		for j in range(nb_phase):
			trafficLights[i][j].attrib["duration"] = str(dur_TLs.pop(0))
		#for j in range(len(dur_TLs[i])):
		#	trafficLights[i][j].attrib["duration"] = dur_TLs[i][j]

	tree.write(file_path)


#get default phase duration from *.net.xml
MAP_PATH = "/home/lin/workspace/GA/GA_Traffic_Optimization/demo_sumo/osm.net.xml"
#MAP_PATH = "/home/zhenyuli/workspace/GA_Traffic_Optimization/demo_sumo/osm.net.xml"
id_TLs, dur_TLs = get_default_duration(MAP_PATH)

best_solution, stats, pop, log = ga(MAP_PATH,dur_TLs,id_TLs)

print("best solution is: ",best_solution[0])
print("best time loss", best_solution[0].fitness.values)

generated:  [47.69966266  2.91880417 51.35369423  4.9225508  32.07024687 11.57720378
 50.06122325  1.02626813 16.37640275  3.44061482 60.          1.41734587
 46.26357205  1.17402928 21.66086359  1.50547495 25.52957496  0.5
 20.20545973  2.17011912 37.87689148  3.00618461]
generated:  [50.14353994  1.96616103 51.59611803  2.25192143 32.40429134 10.97879772
 49.44703781  0.5        17.46484913  3.49571494 58.93999454  0.64440624
 45.48789309  1.04757051 21.1784734   0.5        24.77217098  1.25318227
 19.93235851  2.0600681  36.47491494  3.55044397]
generated:  [49.59929257  2.51022524 52.89511761  4.70868134 32.52355976 10.72509256
 48.52894249  0.5        17.77427683  1.77878645 59.88187846  1.51486396
 44.50949491  0.5        19.49110412  2.67955458 25.27239905  0.6208077
 20.42365519  0.96790157 38.93760215  3.1213268 ]
generated:  [51.2832288   3.91136262 49.62228796  3.10545039 32.22245463  9.53976924
 49.50652584  0.60256348 17.07819254  0.5        59.76067575  0.5
 44.35287611  

##############################################################
##############################################################
avg time loss:  37.44938356164384
##############################################################
##############################################################
eval ind:  [49.030147544523345, 3.245839635152754, 52.27054718319048, 5.042550869238883, 32.26115788823114, 12.320717615206231, 50.45316185614484, 2.366766329931208, 15.517190777070825, 3.318222347724393, 59.92575245677327, 0.9361448920490254, 44.188167898334214, 0.9353591827942722, 19.13243756716219, 1.2732569656647774, 26.503639388357392, 1.0476084886727868, 19.834988531841752, 0.8433689169638579, 37.256199556857304, 3.1357980142048727]
 Retrying in 1 seconds
##############################################################
##############################################################
avg time loss:  36.040239726027394
##############################################################
########################

##############################################################
##############################################################
avg time loss:  33.01569634703196
##############################################################
##############################################################
eval ind:  [51.28322879646863, 3.9113626165704103, 49.62228795553723, 5.277905453557967, 32.222454626823556, 9.539769237871162, 49.50652583555569, 1, 17.078192535568284, 1, 59.760675748242654, 1, 44.352876114320985, 1, 20.138383118524132, 1, 25.80360031867261, 1, 18.572378852911843, 2.4629272882521516, 37.98133251474298, 3.7340250337421352]
 Retrying in 1 seconds
##############################################################
##############################################################
avg time loss:  32.596152968036535
##############################################################
##############################################################
eval ind:  [51.28322879646863, 3.9113626165704103, 50.0488868

##############################################################
##############################################################
avg time loss:  32.041478310502285
##############################################################
##############################################################
eval ind:  [51.28322879646863, 3.9113626165704103, 49.62228795553723, 3.1054503880402247, 32.222454626823556, 9.539769237871162, 49.50652583555569, 1, 17.078192535568284, 1, 59.760675748242654, 1, 44.352876114320985, 1, 20.138383118524132, 1, 25.82097164645074, 1, 18.59533524567781, 2.4629272882521516, 37.98133251474298, 4.231407024353935]
 Retrying in 1 seconds
##############################################################
##############################################################
avg time loss:  31.371227168949773
##############################################################
##############################################################
eval ind:  [51.28322879646863, 3.9113626165704103, 49.6222879

In [2]:
stats

In [3]:
print(stats)

In [5]:
record = stats.compile(pop)
print(record)

{}


In [6]:
log

[{'gen': 0, 'nevals': 10},
 {'gen': 1, 'nevals': 6},
 {'gen': 2, 'nevals': 2},
 {'gen': 3, 'nevals': 2},
 {'gen': 4, 'nevals': 7},
 {'gen': 5, 'nevals': 10}]

In [19]:
gen, nevals = log.select("gen", "nevals")

In [24]:
import plotly.graph_objects as go
import numpy as np

x = np.arange(10)

fig = go.Figure(data=go.Scatter(x=gen, y=nevals, mode='lines+markers'))
fig.update_layout(title='The performance of GA')
fig.show()
